### Models
Looking for `torch` and `keras` pre-trained models.
1. Vision Transformer - Both
2. VGG19 - Both
3. ResNet50 - Both
4. Other Models - MobileNetV2, AlexNet, DenseNet201, DenseNet121, InceptionV3, ResNet50V2, InceptionResNetV2, and Xception

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from tqdm import tqdm

from PIL import Image
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/AJL Team 2/bttai-ajl-2025'

### Data Augmentation
Read the [paper](https://arxiv.org/abs/2104.09957) linked in the Kaggle competition, we can pull some ideas from what they did.

In [5]:
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor()
])

### Load Datasets

In [6]:
train_root_dir = os.path.join(path, 'train', 'train')
train_csv_path = os.path.join(path, 'train.csv')

### Do the HAM10000 DatasetFirst

In [ ]:
from torch.utils.data import Dataset,  DataLoader
import torchvision.transforms as transforms
import os
from PIL import Image

class HAM100000(Dataset):
    def __init__(self, csv_file, part1_dir, part2_dir, transform):
      self.data = pd.read_csv(csv_file)[['image_id', 'dx']]
      self.part1_dir = part1_dir
      self.part2_dir = part2_dir
      self.transform = transform
      self.split_index = 29305
      self.label_map = {  # Convert labels to integers
            'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3,
            'mel': 4, 'nv': 5, 'vasc': 6
        }
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_filename = self.data.iloc[idx, 0]
        label = self.label_map[self.data.iloc[idx, 1]]
        fileSplit = int(self.data.iloc[idx, 0].split('_')[1])
        if fileSplit <= self.split_index:
          img_path = os.path.join(self.part1_dir, img_filename)+'.jpg'
        else:
          img_path = os.path.join(self.part2_dir, img_filename) +'.jpg'
        image = Image.open(img_path).convert("RGB")
        if self.transform:
          image = self.transform(image)

        return image, label

In [ ]:
# Load ImageFolder
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

path_1 = "/content/drive/MyDrive/AJL Team 2/Ham10000/HAM10000_images_part_1"
path_2 = "/content/drive/MyDrive/AJL Team 2/Ham10000/HAM10000_images_part_2"
csvpath = "/content/drive/MyDrive/AJL Team 2/Ham10000/HAM10000_metadata.csv"

dataset_Ham10000 = HAM100000(csvpath, path_1, path_2, transform)

In [ ]:
from torch.utils.data import random_split
#make dataloaders into train and validation split
train_size = int(0.8 * len(dataset_Ham10000))
val_size = len(dataset_Ham10000) - train_size
train_dataset_HAM10000, val_dataset_HAM10000 = random_split(dataset_Ham10000, [train_size, val_size])

In [ ]:
# Load pretrained ViT model
model = models.vit_b_16(weights="IMAGENET1K_V1")

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 160MB/s]


In [ ]:
# Replace classifier head (FC layer)
model.heads.head = nn.Linear(model.heads.head.in_features, 7)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Freeze all parameters first
"""
for param in model.parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if "encoder.layers.encoder_layer_" in name:  # Match correct layer naming
        layer_num = int(name.split("encoder.layers.encoder_layer_")[-1].split(".")[0])  # Extract layer index
        if layer_num >= 8:  # Unfreeze last 4 blocks
            param.requires_grad = True
        elif "heads.head" in name:  # Unfreeze final layer head
         param.requires_grad = True"""
         #dont freeze anything


'\nfor param in model.parameters():\n    param.requires_grad = False\n\nfor name, param in model.named_parameters():\n    if "encoder.layers.encoder_layer_" in name:  # Match correct layer naming\n        layer_num = int(name.split("encoder.layers.encoder_layer_")[-1].split(".")[0])  # Extract layer index\n        if layer_num >= 8:  # Unfreeze last 4 blocks\n            param.requires_grad = True\n        elif "heads.head" in name:  # Unfreeze final layer head\n         param.requires_grad = True'

In [7]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 14.1 MB/s eta 0:00:00


In [8]:
import torch
from torcheval.metrics import MulticlassAccuracy, MulticlassF1Score
from tqdm import tqdm

# Function to train and validate model
def train_model(model, train_data, val_data, batch_size, criterion, optimizer, num_classes, device, save_path, num_epochs=10, scheduler=None):

    # Create DataLoaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    model.to(device)

    best_val_f1 = float('-inf')  # Initialize best F1 score as very low

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        scaler = torch.amp.GradScaler(device)

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        accuracy_metric = MulticlassAccuracy(num_classes=num_classes, device=device)
        f1_metric = MulticlassF1Score(num_classes=num_classes, average="macro", device=device)

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Update metrics
                accuracy_metric.update(outputs, labels)
                f1_metric.update(outputs, labels)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = accuracy_metric.compute().item()
        val_f1 = f1_metric.compute().item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")

        # Save best model based on F1 score
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            save_model(model, save_path)

        # Step the scheduler if provided
        if scheduler:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(avg_val_loss)  # ReduceLROnPlateau needs validation loss
            else:
                scheduler.step()  # Other schedulers just step normally

# Function to save model
def save_model(model, path="/content/drive/MyDrive/AJL Team 2/HAM1000model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")


In [ ]:
# Loss and Optimizer
#tbh probably just gonna have this run until it break or something
criterion = nn.CrossEntropyLoss()
#give it an optimizer

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


train_model(model, train_dataset_HAM10000, val_dataset_HAM10000, 64, criterion, optimizer, 7, device, '/content/drive/MyDrive/AJL Team 2/VITLHAM1000model.pth', 10)

## After pretrain on the augmented dataset

In [9]:
#create dataloader for this
from torchvision.datasets import ImageFolder
class ImageFolderWithColor(Dataset):
    def __init__(self, image_folder):
        self.image_folder = image_folder
        self.image_paths, self.labels = zip(*self.image_folder.samples)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        # Extract filename from the path
        filename = os.path.basename(img_path)

        # Load image and apply transformations
        image, _ = self.image_folder[idx]

        return image, label

In [10]:
#add in a bunch of transforms
from torchvision.transforms.v2 import ElasticTransform
"""transform = transforms.Compose([transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(4/5, 5/4)),  # Keeps most of the lesion
    transforms.RandomHorizontalFlip(p=0.7),  # Flip images randomly
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.3, hue=0.05),  # Preserve color, minimal hue change
    transforms.RandomRotation(degrees=10),  # Small rotations
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1, 0.5)),  # Slight blur
    transforms.ToTensor()  # Convert to tensor (NO NORMALIZATION, does not make sense for this task!)
])"""
#different transformations
transform = transforms.Compose([
    # Random crop while maintaining lesion structure
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(4/5, 5/4)),

    # Horizontal flip
    transforms.RandomHorizontalFlip(p=0.7),

    # Color jitter for brightness, contrast, saturation, hue
    transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),

    # Random rotation
    transforms.RandomRotation(degrees=20),

    # Slight Gaussian blur
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 0.8)),

    # Elastic transformation (simulates deformations, helps robustness)
    ElasticTransform(alpha=30.0),

    # Convert to tensor (NO NORMALIZATION here if it's task-specific)
    transforms.ToTensor()
])

#we are just going to work with the original dataset for now
path = "/content/drive/MyDrive/AJL Team 2/bttai-ajl-2025/train/train"
# Load dataset from the correct directory
dataset_pre = ImageFolder(root=path, transform=transform)

# Create the custom dataset
dataset = ImageFolderWithColor(dataset_pre)

# Load into DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [11]:
from torch.utils.data import random_split
# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [12]:
##load in pretrained model then replace the fc with the correct size
finalModel = models.vit_b_16(weights="IMAGENET1K_V1")

# Replace classifier head (FC layer)
finalModel.heads.head = nn.Linear(finalModel.heads.head.in_features, 7)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
finalModel = finalModel.to(device)

# Load the saved weights
finalModel.load_state_dict(torch.load("/content/drive/MyDrive/AJL Team 2/VITHAM1000model.pth", map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))

finalModel.heads.head = nn.Linear(finalModel.heads.head.in_features, 21) #switch out to correct size final layer

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 180MB/s]


In [13]:
# Freeze all parameters first
for param in finalModel.parameters():
    param.requires_grad = False

for name, param in finalModel.named_parameters():
    if "encoder.layers.encoder_layer_" in name:  # Match correct layer naming
        layer_num = int(name.split("encoder.layers.encoder_layer_")[-1].split(".")[0])  # Extract layer index
        if layer_num >= 4:  # Unfreeze last 4 blocks
            param.requires_grad = True
        elif "heads.head" in name:  # Unfreeze final layer head
         param.requires_grad = True

# Move model to GPU
finalModel.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [14]:
#now retrain the original model using the data we were given and see how it does

#reduce lr if same for 2 epochs
from torch.optim.lr_scheduler import ReduceLROnPlateau

#give it an optimizer, adding ab it of weight decay
#seems to overfitt give it a bit more weigth decay
optimizer = torch.optim.Adam(finalModel.parameters(), lr=1e-4, weight_decay=5e-5)
#use a learning rate schedular

#give patience of 3 and a minimum lr so that it always keeps learning
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, min_lr=1e-7)
#give a learning rate schedular
#scheduler = CosineAnnealingLR(optimizer, T_max=20)  # 10 epochs total

/drive/MyDrive/AJL Team 2/VIT_train_ham_first_20epochs_plateulr_patience_2_augmentedset8Layers',device=device, scheduler = scheduler)

  0%|          | 0/36 [00:00<?, ?it/s]<ipython-input-8-12a6e13e4f55>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 36/36 [15:11<00:00, 25.31s/it]


Epoch [1/20], Train Loss: 2.3777, Val Loss: 1.9190, Val Acc: 0.3951, Val F1: 0.2747
Model saved to /content/drive/MyDrive/AJL Team 2/VIT_train_ham_first_20epochs_plateulr_patience_2_augmentedset8Layers


100%|██████████| 36/36 [02:41<00:00,  4.50s/it]


Epoch [2/20], Train Loss: 1.5227, Val Loss: 1.5900, Val Acc: 0.5140, Val F1: 0.4188
Model saved to /content/drive/MyDrive/AJL Team 2/VIT_train_ham_first_20epochs_plateulr_patience_2_augmentedset8Layers


100%|██████████| 36/36 [02:44<00:00,  4.57s/it]


...

Epoch [18/20], Train Loss: 0.0326, Val Loss: 1.5044, Val Acc: 0.6259, Val F1: 0.5678


100%|██████████| 36/36 [02:41<00:00,  4.48s/it]


Epoch [19/20], Train Loss: 0.0323, Val Loss: 1.4894, Val Acc: 0.6416, Val F1: 0.5841
Model saved to /content/drive/MyDrive/AJL Team 2/VIT_train_ham_first_20epochs_plateulr_patience_2_augmentedset8Layers


100%|██████████| 36/36 [02:39<00:00,  4.43s/it]


Epoch [20/20], Train Loss: 0.0305, Val Loss: 1.4799, Val Acc: 0.6259, Val F1: 0.5629


In [15]:
TestModel = models.vit_b_16(weights="IMAGENET1K_V1")
#need to change the fc to finetune it
num_ftrs = TestModel.heads.head.in_features
TestModel.heads.head = nn.Linear(num_ftrs, 21)  # 7 classes for HAM10000

# Load the saved weights
TestModel.load_state_dict(torch.load('/content/drive/MyDrive/AJL Team 2/VIT_train_ham_first_20epochs_plateulr_patience_2_augmentedset8Layers', map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
TestModel.to(device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [16]:
#now need to get testing labels to submit
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_hash = row["md5hash"]

        img_path = os.path.join(self.root_dir, img_hash + ".jpg")

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image {img_hash}.jpg not found in {img_path}")

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, img_hash  # return hash to map it to a label in submission csv

# same transformations as training
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
path_test = '/content/drive/MyDrive/AJL Team 2/bttai-ajl-2025'

# path to test CSV and test image folder
test_csv_path = os.path.join(path_test, 'test.csv')
test_root_dir = os.path.join(path_test, 'test', 'test')

# load test dataset
test_dataset = TestDataset(test_csv_path, test_root_dir, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

print(f"Total test samples: {len(test_dataset)}")

Total test samples: 1227


In [17]:
def predict_and_save(model, test_loader, device, idx_to_class):
    model.eval

    predictions = []

    with torch.no_grad():
        for images, img_hashes in tqdm(test_loader, desc="Generating Predictions"):
            images = images.to(device)

            # predict
            outputs = model(images)

            # Get predicted class index
            _, predicted = torch.max(outputs, 1)

            # Convert index to class label
            predicted_labels = [idx_to_class[idx.item()] for idx in predicted]

            # Store results
            for img_hash, label, in zip(img_hashes, predicted_labels):
                predictions.append({"md5hash": img_hash, "label": label})
    submission_df = pd.DataFrame(predictions)
    return submission_df

In [18]:

# Map indices to class labels
idx_to_class = {idx: cls for cls, idx in dataset_pre.class_to_idx.items()}

# Run predictions and save
submission_df=predict_and_save(TestModel, test_loader, device, idx_to_class)

Generating Predictions: 100%|██████████| 77/77 [08:14<00:00,  6.43s/it]


In [19]:
submission_df.head()
#Save to CSV
submission_df.to_csv(path_test + "/ViTLPreTrainOnHamInLoaderAugmentPatience2AugemntedSetTrain8LayersHAMMALLlessWEightDecat.csv", index=False)